# Context Managers in Python

---

__Elliott Forney - 2020__

Context managers are a powerful construct in Python that allow an object to create a temporary "context" with initialization and teardown logic that are automatically called when the context is entered and exited, respectively.

As we will see, context managers allow for better resource management and for the responsibility to manage resources to be placed inside of a library, rather than having that burdon be placed on the caller.  This is similar to the notion of Resource Allocation is Instantiation (RAII) in C++ and other languages, but is somewhat finer grained and can be decoupled from the actual construction and deletion of a python object.  Note that since python has function-level scoping and automatic memory management, context managers allow us to define narrow and well-defined sections of code over which a resource is available.

## Motivation

Let us begin with some motivating examples.  Below, we create a file handle (over this notebook) and manually open and close the file handle after we are done with it.  Note that it is important to close the file handle or else it will remain open indefinitely, which may constitute a resource leak and cause a program that opens many files to eventually crash.

In [1]:
fh = open('context_managers_in_python.ipynb', mode='r')
data = fh.read()
fh.close()

print(data[:120])

{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Context Managers in Python\n",
  


Although manually closing the file handle gets the job done, this approach has an important problem:

* If an exception is raised before `close` is called, the file handle will never be closed!!

### Handling with traditional exception semantics

Ordinary exception handling constructs allow us to handle the problem of ensuring that `close` is called by adding a `finally` clause.  In the example below, `close` will be called regardless of whether or not an exception is raised.  Specifically, the code block inside of the `finally` clause is called whenever flow control leaves the `try` block.

In [2]:
try:
    fh = open('context_managers_in_python.ipynb', mode='r')
    data = fh.read()

finally:
    fh.close()
    
print(data[:120])

{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Context Managers in Python\n",
  


Although this approach does ensure that `close` is called, it also introduces several important problems to consider.

* Layers of exception handling.  Not bad in this example, but but gets worse (we'll see below).
* Caller is responsible for cleaning up resources, should be job of library!
* What if the code in the `try` block only partially completes?  May need to have significant logic in the `finally` block in order to ensure proper state.

### Context managers for managing resources

Context managers offer an alternative approach where a temporary context is created and the responsibility is placed on the called library to ensure that resources are created when the context is entered and then cleaned up when the context exits, including the case where control flow exits the context because an exception was raise or because a function calls `return`.

Context managers use the `with object` syntax where the `with` keyword indicates the beginning of the context and where `object` is any python object that implementes the context manager interface, which we will describe in more detail below.  Optionally, the `with object as reference` syntax can also be used to get a reference to an object that is returned by the context manager interface.  This is often simply a reference to the `object`, but not always.  Again, this will be explained in more detail once we see how the context manager interface is implemented.

Note that the boundaries of the context are defined by indentation, of course, with initialization occuring when the `with` statement is called and teardown occuring when flow control exits the indented block.

In the example below, which is familiar to most python programmers, we use the context manager interface provided by `open` in order to ensure that our file is opened and closed, without requiring `try/finally` clauses.

In [3]:
with open('context_managers_in_python.ipynb', mode='r') as fh:
    data = fh.read()
    
print(data[:120])

{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Context Managers in Python\n",
  


* No opportunity for mistakes by the caller, can't forget the `finally`
* Resource cleanup handled within the library, caller does nothing
* Only the library needs to handle ensuring consistent state when the code only executes partially
* Resource cleanup can __change__ inside the library without breaking the API
* Fewer lines of code, elegant feeling

### Multiple context managers on one line

By the way, context managers can be nested and we can also place multiple context managers on a single line!

In [4]:
infile = 'context_managers_in_python.ipynb'
outfile = 'test.txt'

with open(infile, mode='r') as in_fh, open(outfile, mode='w') as out_fh:
    data = in_fh.read()
    out_fh.write(data)

## A more sophisticated example: socket exception spaghetti

This all sounds great, but it's difficult to grasp exactly how valuable this is until we see it in practice in a more sophisticated example.

In order to illustrate this, let's examine a contrived example where we have multiple exception handling clauses around both socket and file IO.

In this example, we'll try to read my homepage over HTTP using raw sockets and then write it to a file.  Along the way, we'll add some useful information to the raised exceptions.  Adding this kind of information is often useful for logging and debugging purposes.

In [5]:
import socket

In [6]:
host, port = 'www.elliottforney.com', 80
request = b'GET / HTTP/1.1\r\nHost: www.elliottforney.com\r\nAccept: text/html\r\n\r\n'
outfile = 'response.txt'

In [7]:
# open a new socket, ensure it's closed on exception
try:
    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    
    # try to connect, add info to the exception on failure
    try:
        s.connect((host, port))
    except OSError as ex:
        raise OSError(f'<E001> failed to connect: {str(ex)}') from ex
    
    # try to sent HTTP request, add info to exception on failure
    try:
        s.send(request)
    except OSError as ex:
        raise OSError(f'<E002> failed to send request: {str(ex)}') from ex
    
    # open a file to write the response, ensure it's closed on exception
    try:
        fh = open(outfile, mode='wb')
    
        # while there's data to read
        while True:
            # get 512 bytes and write to the file
            result = s.recv(512)
            if not result:
                break
        
            fh.write(result)
    
    # close the file handle
    finally:
        fh.close()

# close the socket
finally:
    s.close()
    
# re-open the file for printing
try:
    fh = open(outfile, mode='r')
    print(fh.read())

# close the file handle
finally:
    fh.close()

HTTP/1.1 301 Moved Permanently
Date: Wed, 14 Oct 2020 19:45:05 GMT
Server: Apache
Location: https://www.elliottforney.com/
Content-Length: 238
Content-Type: text/html; charset=iso-8859-1

<!DOCTYPE HTML PUBLIC "-//IETF//DTD HTML 2.0//EN">
<html><head>
<title>301 Moved Permanently</title>
</head><body>
<h1>Moved Permanently</h1>
<p>The document has moved <a href="https://www.elliottforney.com/">here</a>.</p>
</body></html>



Although this certainly works fine, and is similar to how exception handling works in languages like Java and C++, it is lengthy, prone to failures and requires the caller to know how to clean things up.

Note, however, that the `socket` interface in Python also adheres to the context manager interface, which means that we can create a context, just like we did with `open`.  This example below shows how the code differs when this approach is taken.

In [8]:
# open a socket to the desired host
with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:   
    
    # try to connect, add info to the exception on failure
    try:
        s.connect((host, port))
    except OSError as ex:
        raise OSError(f'<E001> failed to connect: {str(ex)}') from ex
        
    # try to send the request, add info to the exception on failure
    try:
        s.send(request)
    except OSError as ex:
        raise OSError(f'<E002> failed to send request: {str(ex)}') from ex
    
    # open a file for writing the response
    with open(outfile, mode='wb') as fh:
        # while there's data to read
        while True:
            # get 512 bytes and write to the file
            result = s.recv(512)
            if not result:
                break
        
            fh.write(result)

# re-open the file for printing
with open(outfile, mode='r') as fh:
    print(fh.read())

HTTP/1.1 301 Moved Permanently
Date: Wed, 14 Oct 2020 19:45:10 GMT
Server: Apache
Location: https://www.elliottforney.com/
Content-Length: 238
Content-Type: text/html; charset=iso-8859-1

<!DOCTYPE HTML PUBLIC "-//IETF//DTD HTML 2.0//EN">
<html><head>
<title>301 Moved Permanently</title>
</head><body>
<h1>Moved Permanently</h1>
<p>The document has moved <a href="https://www.elliottforney.com/">here</a>.</p>
</body></html>



Notice how much more concise the code is now!  Also, the user no longer has to worry about closing sockets or file handles because this is done automatically, and within the library, when the context manager exits.

We do, however, still have to add `try`/`catch` statements around our `.connect` and `.send` statements in order to augment the exception messages.  Since we are trying to move this sort of logic away from the caller and into a library, not that we could also create a simple object-oriented wrapper around the `socket` class in order to add our extra information.  This allows us to create a short, generic interface where the caller doesn't have to do any exception handling.

In [9]:
class LoggedSocket(socket.socket):
    def connect(self, *args, **kwargs):
        try:
            return super().connect(*args, **kwargs)
        except OSError as ex:
            raise OSError(f'<E001> failed to connect: {str(ex)}') from ex
            
    def send(self, *args, **kwargs):
        try:
            return super().send(*args, **kwargs)
        except OSError as ex:
            raise OSError('<E002> failed to send request: {str(ex)}') from ex

Now, the caller can focus on the goal that they are trying to achieve, without having a spaghetti mess of boilerplate `try`/`catch` statements.

In [10]:
# open a socket with extra debugging information
with LoggedSocket(socket.AF_INET, socket.SOCK_STREAM) as s:    
    # connect and send the request
    s.connect((host, port))
    s.send(request)
    
    # open a file for writing
    with open(outfile, mode='wb') as fh:
        # while there's still data to read
        while True:
            # get 512 bytes and write to the file
            result = s.recv(512)
            if not result:
                break
        
            fh.write(result)

# re-open the file and print its contents
with open(outfile, mode='r') as fh:
    print(fh.read())

HTTP/1.1 301 Moved Permanently
Date: Wed, 14 Oct 2020 19:45:15 GMT
Server: Apache
Location: https://www.elliottforney.com/
Content-Length: 238
Content-Type: text/html; charset=iso-8859-1

<!DOCTYPE HTML PUBLIC "-//IETF//DTD HTML 2.0//EN">
<html><head>
<title>301 Moved Permanently</title>
</head><body>
<h1>Moved Permanently</h1>
<p>The document has moved <a href="https://www.elliottforney.com/">here</a>.</p>
</body></html>



Although this is certainly a contrived example, it demonstrates how a very long piece of code with many `try`/`catch` statements can be simplified dramatically by using context managers.  I hope that it also illustrates how context managers can be used to move resource management logic from boilerplate code, to something that is managed from within the library that is being called.

## How do context managers work?

Now that we've seen how to use context managers effectively, let's see how they work internally and consider how to build our own customer context managers.

Context managers start with an arbitrary python class and require only that the class implements the context manager interface by providing two methods named `__enter__` and `__exit__`.  These methods are, not surprisingly, called when the context manger enters and exits, respectively.

Note that the constructor for the object is separate from `__enter__`.  Often,  `__enter__` will simply return a reference to `self`, but we will see some examples later where a context manager might want to return a reference to something different.

The `__exit__` method takes a number of arguments that are related to exception handling.  These arguments tell the function if an exception was raised from within the context and provides information about the exception so that resources can be cleaned up accordingly.

Note that __if an exception is thrown, it is still re-raised immediately after `__exit__` returns__.  This allows the context manager to clean up appropriately while also allowing the caller to catch and handle the error when necessary.

In the example below, we see a very simple context manager that prints a debug message at each step.

In [11]:
class ContextDemo:
    def __init__(self):
        print('initializing object')
    
    def __enter__(self):
        print('setting up context')
        return self
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        if exc_type is not None:
            print(f'caught exception: {exc_type.__name__}: {exc_val!s}')

        print('doing some cleanup')

In the case when no exception is raised, our context manager simply prints a message in the constructor, from `__enter__`, from within the context and then, finally, inside `__exit__`.

In this case, the arguments to `__exit__` are all `None` since no exception was raised.

In [12]:
with ContextDemo():
    print('do some work in the context')

initializing object
setting up context
do some work in the context
doing some cleanup


If an exception is raised from within the context, then a reference to the raised exception is passed to the `.__exit__` method and a corresponding message is printed to the console.

Again, note that the exception is then re-raised, so the exception is further propigated to our Jupyter notebook.

In [13]:
with ContextDemo():
    raise RuntimeError('some exception')

initializing object
setting up context
caught exception: RuntimeError: some exception
doing some cleanup


RuntimeError: some exception

As we mentioned previously, the construction of the context manager is also separated from the `.__enter__` step.  This means that the corresponding object can actually be created *before* entering the context manager and multiple contexts can even be created for the same object!

In [ ]:
# create the `ContextDemo` object
context_object = ContextDemo()

# start the first context
print('=======')
with context_object:
    print('doing some work')
    
# start a second context with the same object!
print('=======')
with context_object:
    print('doing some work')

### A demonstration wrapper around `open`

In the next example of a context manager, we'll simply pretend that the `open` function does not already implement the context manager interface and show how it could be done.

First, the constructor takes a file name and any additional arguments to pass to `open`.  We don't call `open` yet, however, because we'll want to be able to create multiple contexts using a single `OpenWrapper` object, i.e., open the file multiple times, as was demonstrated in the previous example.

We then implement an `__enter__` method that calls the `open` function and returns a reference to the file handle that it returns.  This means that if we write `with OpenWraper(...) as fh` then `fh` is a reference to the handle returned by `open` and *not* to an `OpenWrapper` object.

Finally, we implement an `__exit__` method that simply calls `close` on the file descriptor.  This should be done regardless of whether or not an exception was raised, just like with a `finally` clause.

In [ ]:
class OpenWrapper:
    '''Wrapps calls to `open` with a simple context manager.'''
    def __init__(self, filename, *args, **kwargs):
        '''Construct a new `OpenWrapper`, but don't open anything yet.'''
        self.filename, self.args, self.kwargs = filename, args, kwargs
        self.fh = None
    
    def __enter__(self):
        '''Start a context by opening the file and returning a file handle.'''
        self.fh = open(self.filename, *self.args, **self.kwargs)
        return self.fh
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        '''Exit the context by closing the file handle.'''
        self.fh.close()

This simple context manager can now be used almost exactly like the context manager interface that is already provided by the `open` function in the standard Python API.

In [ ]:
with OpenWrapper('context_managers_in_python.ipynb') as fh:
    data = fh.read()
    
print(data[:120])

### Implementing a timer class with a context manager

It is important to note that context managers have __many__ uses other than handling exceptions.  Any time that there are resources to be managed over a defined period of time and where we want the called library to handle said resources, a context manager may be an appropriate choice.

In this example, we demonstrate how to use context managers to implement a simple timer that can be called over multiple trial runs of a section of code.

In [ ]:
from time import time

class Timer:
    def __init__(self):
        self.cumulative_time = 0.0
        self.start_time = 0.0
        self.ntrial = 0
    
    def __enter__(self):
        self.ntrial += 1
        self.start_time = time()
        return self
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        if exc_type is not None:
            print('not timing errored trial')
            return
        
        self.cumulative_time += time() - self.start_time
        
    def __repr__(self):
        return str(round(self.cumulative_time / self.ntrial, 4))

In [ ]:
import numpy as np
n = 2048

In [ ]:
a = np.random.random((n, n))
b = np.random.random((n, n))

with Timer() as timer:
    c = a @ b
    
print('time:', timer)

In [ ]:
timer = Timer()

for trial in range(10):
    print('trial:', trial)
    a = np.random.random((n, n))
    b = np.random.random((n, n))
    
    with timer:
        c = a @ b

        
print('average time:', timer)

In [ ]:
timer = Timer()

for trial in range(10):
    print('trial:', trial)
    a = np.random.random((n, n))
    b = np.random.random((n, n))
    
    try:
        with timer:
            if trial == 3:
                raise RuntimeError('something broke')
            
            c = a @ b
    except:
        pass
        
print('average time:', timer)

## `contextlib`

The `contextlib` module is part of the standard python API and contains many useful functions that either are context managers or are usful for creating or utilizing context managers.

In [ ]:
import contextlib

### Suppressing exceptions

In [ ]:
import os

try:
    os.remove('path/does/not/exist')
except:
    pass

In [ ]:
with contextlib.suppress(FileNotFoundError):
    os.remove('path/does/not/exist')

By the way, we just did something like this in our previous example and `contextlib.suppress` would have been useful here.

In [ ]:
timer = Timer()

for trial in range(10):
    print('trial:', trial)
    a = np.random.random((n, n))
    b = np.random.random((n, n))
    
    with contextlib.suppress(RuntimeError), timer:
        if trial == 3:
            raise RuntimeError('something broke')
            
        c = a @ b

print('average time:', timer)

### Redirecting `stderr` or `stdout`

In [ ]:
with open('help.txt', mode='w') as fh:
    with contextlib.redirect_stdout(fh):
        print('hello world!')
        print()
        help(os.remove)

In [ ]:
with open('help.txt', mode='r') as fh:
    print(fh.read())

In [ ]:
with open('help.txt', mode='w') as fh, contextlib.redirect_stdout(fh):
    print('hello world!')
    print()
    help(os.remove)

In [ ]:
with open('help.txt', mode='r') as fh:
    print(fh.read())